[View in Colaboratory](https://colab.research.google.com/github/maddy46/mlblr4a_experiments/blob/master/4b_01.ipynb)

In [40]:
# https://keras.io/
!pip install -q keras
import keras

In [41]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [42]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [43]:
# Hyperparameters
batch_size = 35
num_classes = 10
epochs = 35
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [44]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print (img_height)
print (img_width)

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

32
32


In [45]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [46]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [47]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [48]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
print(channel)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
#First_Transition = add_transition(First_Block, num_filter, dropout_rate)

#Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
#Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

#Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
#Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

#Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
#output = output_layer(Last_Block)
output = output_layer(First_Block)


3


In [49]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_27 (Conv2D)              (None, 32, 32, 12)   324         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_27 (BatchNo (None, 32, 32, 12)   48          conv2d_27[0][0]                  
__________________________________________________________________________________________________
activation_27 (Activation)      (None, 32, 32, 12)   0           batch_normalization_27[0][0]     
__________________________________________________________________________________________________
conv2d_28 

In [50]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/35
23940/50000 [=============>................] - ETA: 1:04 - loss: 1.7237 - acc: 0.4469

50000/50000 [==============================] - 129s 3ms/step - loss: 1.4577 - acc: 0.5159 - val_loss: 1.3500 - val_acc: 0.5442
Epoch 2/35
 5845/50000 [==>...........................] - ETA: 1:44 - loss: 1.0093 - acc: 0.6503

49980/50000 [============================>.] - ETA: 0s - loss: 0.9736 - acc: 0.6622

50000/50000 [==============================] - 126s 3ms/step - loss: 0.9735 - acc: 0.6622 - val_loss: 1.1739 - val_acc: 0.6157
Epoch 3/35
15645/50000 [========>.....................] - ETA: 1:21 - loss: 0.8185 - acc: 0.7166

50000/50000 [==============================] - 126s 3ms/step - loss: 0.8241 - acc: 0.7153 - val_loss: 1.3133 - val_acc: 0.5950
Epoch 4/35
 2695/50000 [>.............................] - ETA: 1:51 - loss: 0.6832 - acc: 0.7607

...

50000/50000 [==============================] - 126s 3ms/step - loss: 0.7291 - acc: 0.7482 - val_loss: 0.9326 - val_acc: 0.6899
Epoch 5/35
15645/50000 [========>.....................] - ETA: 1:20 - loss: 0.6187 - acc: 0.7909

50000/50000 [==============================] - 126s 3ms/step - loss: 0.6469 - acc: 0.7764 - val_loss: 0.8904 - val_acc: 0.7034
Epoch 6/35
 2695/50000 [>.............................] - ETA: 1:51 - loss: 0.5033 - acc: 0.8249

49980/50000 [============================>.] - ETA: 0s - loss: 0.5725 - acc: 0.8011

50000/50000 [==============================] - 126s 3ms/step - loss: 0.5724 - acc: 0.8011 - val_loss: 1.1214 - val_acc: 0.6554
Epoch 7/35
15645/50000 [========>.....................] - ETA: 1:20 - loss: 0.4744 - acc: 0.8369

47075/50000 [===========================>..] - ETA: 6s - loss: 0.5099 - acc: 0.8213

In [37]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 9s 853us/step
Test loss: 0.9560066279411316
Test accuracy: 0.7187


In [38]:
# Save the trained weights in to .h5 format
#model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [39]:
#from google.colab import files

#files.download('DNST_model.h5')